In [ ]:

!pip install roboflow
!pip install ultralytics -q
!pip install gradio -q
!pip install opencv-python matplotlib


from google.colab import drive
import os

drive.mount('/content/drive')


PROJECT_DIR = '/content/drive/MyDrive/LogoRecognition'
os.makedirs(PROJECT_DIR, exist_ok=True)


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

model_path = f"{PROJECT_DIR}/train_run/weights/best.pt"

print(f"Đang tải mô hình từ: {model_path}")
model = YOLO(model_path)

test_image_path = f"{PROJECT_DIR}/{dataset_folder_name}/test/images/9080161_jpg.rf.3ad318727baa90314970203b83f93a99.jpg"

if os.path.exists(test_image_path):
    results = model(test_image_path)

    for result in results:
        result.show()

        save_path = f"{PROJECT_DIR}/result_prediction.jpg"
        result.save(filename=save_path)
        print(f"Đã lưu ảnh kết quả tại: {save_path}")
else:
    print(f"Không tìm thấy file ảnh: {test_image_path}")
    print("Vui lòng vào Drive kiểm tra thư mục 'test/images' để lấy đúng tên file ảnh.")

In [ ]:
import gradio as gr

def predict_logo(image_path): results = model.predict(source=image_path, save=True, exist_ok=True)

result_path = results[0].save_dir
output_image_path = os.path.join(result_path, os.path.basename(image_path))

return output_image_path
iface = gr.Interface( fn=predict_logo, inputs=gr.Image(type="filepath", label="Tải lên ảnh cần nhận diện Logo"), outputs=gr.Image(label="Kết quả Phát hiện Logo"), title="Demo Phát hiện và Phân loại Logo bằng YOLOv8" )

iface.launch(share=True)

In [ ]:
from roboflow import Roboflow
import shutil

rf = Roboflow(api_key="PXwQ4NqbMHWJHN6A0VTl")
project = rf.workspace("vu-quoc-minh-dang").project("flickrlogos32")
version = project.version(1)
dataset = version.download("yolov8")

dataset_folder_name = "FlickrLogos32-1"

source_dir = f"/content/{dataset_folder_name}"
dest_dir = f"{PROJECT_DIR}/{dataset_folder_name}"

if os.path.exists(source_dir):
    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)

    shutil.move(source_dir, dest_dir)
    print(f"Đã lưu Dataset vào: {dest_dir}")
else:
    print("Không tìm thấy folder tải về, hãy kiểm tra lại tên folder.")


import yaml

yaml_path = f"{dest_dir}/data.yaml"

with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# Sửa đường dẫn thành tuyệt đối
data['path'] = dest_dir
data['train'] = 'train/images'
data['val'] = 'valid/images'
data['test'] = 'test/images'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f)


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

yaml_file = f"{PROJECT_DIR}/{dataset_folder_name}/data.yaml"

results = model.train(
    data=yaml_file,
    epochs=50,
    imgsz=640,
    project=PROJECT_DIR,
    name='train_run'
)